In [180]:
import json
import pandas as pd
import requests

In [181]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [182]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [183]:
with open("../shared-folder-gald/keys1.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [184]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [185]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [186]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [187]:
access_token = get_access_token(client_key, client_secret)

In [188]:
liked_videos = []
for username in usernames[12989:13868]:
    liked_videos.append(get_liked_info(username, access_token))

In [189]:
combined = {}
for i, username in enumerate(usernames[12989:13868]):
    combined[username] = liked_videos[i]

In [190]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)

                username                   id             creator
0     atuwerhossainridoy  7451420779469688080    mdzahidhasan1636
1     atuwerhossainridoy  7453144018323639568    mdzahidhasan1636
2     atuwerhossainridoy  7454618071815294209    mdzahidhasan1636
3     atuwerhossainridoy  7449835435552935184    mdzahidhasan1636
4     atuwerhossainridoy  7450505456025390353    mdzahidhasan1636
...                  ...                  ...                 ...
1068       maikengreimel  7467861092786638102         janes.wonda
1069       maikengreimel  7465696976609266966  desperatebookwives
1070       maikengreimel  7466856348878212374         ninagreimel
1071       maikengreimel  7451570066488921366        sandrasbooks
1072       maikengreimel  7429042977038732587          bookish.hb

[1073 rows x 3 columns]


In [191]:
likes_df.to_csv("liked_videos.csv", mode="a", index=False, header=False)